In [1]:
# Load modules
import hddm
import tensorflow as tf
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import scipy as scp
import psutil
from time import time
from copy import deepcopy
import os
import pickle
import argparse
import yaml as yml
from multiprocessing import Pool
from functools import partial


In [ ]:
hddm.simulators.simulator_single_subject()

In [3]:
help(hddm.simulators.simulator_single_subject)

Help on function simulator_single_subject in module hddm.simulators.hddm_dataset_generators:

simulator_single_subject(parameters=(0, 0, 0), p_outlier=0.0, max_rt_outlier=10.0, model='angle', n_samples=1000, delta_t=0.001, max_t=20, bin_dim=None, bin_pointwise=False)
    Generate a hddm-ready dataset from a single set of parameters
    
    :Arguments:
        parameters: dict, list or numpy array
            Model parameters with which to simulate. Dict is preferable for informative error messages.
            If you know the order of parameters for your model of choice, you can also directly supply a
            list or nump.array which needs to have the parameters in the correct order.
        p_outlier: float between 0 and 1 <default=0>
            Probability of generating outlier datapoints. An outlier is defined
            as a random choice from a uniform RT distribution
        max_rt_outlier: float > 0 <default=10.0>
            Using max_rt_outlier (which is commonly define

In [6]:
help(hddm.simulators.hddm_dataset_generators)

Help on module hddm.simulators.hddm_dataset_generators in hddm.simulators:

NAME
    hddm.simulators.hddm_dataset_generators

FUNCTIONS
    ddm(...)
    
    ddm_flexbound(...)
    
    ddm_flexbound_mic2_adj(...)
    
    ddm_flexbound_par2(...)
    
    ddm_flexbound_seq2(...)
    
    ddm_sdv(...)
    
    full_ddm(...)
    
    full_ddm_vanilla(...)
    
    hddm_preprocess(simulator_data=None, subj_id='none', keep_negative_responses=False, add_model_parameters=False, keep_subj_idx=True)
        Takes simulator data and turns it into HDDM ready format.
        
        :Arguments:
            simulator_data: tuple
                Output of e.g. the hddm.simulators.basic_simulator function.
            subj_id: str <default='none'>
                Subject id to attach to returned dataset
            keep_negative_responses: bool <default=False>
                Whether or not to turn negative responses into 0
            add_model_parameters: bool <default=False>
                Whet

In [8]:
help(hddm.simulators.make_parameter_vectors_nn)

Help on function make_parameter_vectors_nn in module hddm.simulators.hddm_dataset_generators:

make_parameter_vectors_nn(model='angle', param_dict=None, n_parameter_vectors=10)
    Generates a (number of) parameter vector(s) for a given model.
    
    :Arguments:
    
        model: str <default='angle'>
            String that specifies the model to be simulated.
            Current options include, 'angle', 'ornstein', 'levy', 'weibull', 'full_ddm'
        param_dict: dict <default=None>
            Dictionary of parameter values that you would like to pre-specify. The dictionary takes the form (for the simple examples of the ddm),
            {'v': [0], 'a': [1.5]} etc.. For a given key supply either a list of length 1, or a list of
            length equal to the n_parameter_vectors argument supplied.
        n_parameter_vectors: int <default=10>
            Nuber of parameter vectors you want to generate
    
    Return: pandas.DataFrame
            Columns are parameter names an

In [13]:
param_sets = hddm.simulators.make_parameter_vectors_nn(model = 'ddm',
                                          n_parameter_vectors = 256)

                                          

In [19]:
model_data = hddm.simulators.simulator_single_subject(model = 'ddm', 
                                                      parameters = param_sets.loc[0, :][['v', 'a', 'z', 't']].values, 
                                                      p_outlier = 0.0,
                                                      n_samples = 1000)

Model:  ddm
Parameters needed:  ['v', 'a', 'z', 't']
[-1.87642118  2.35665914  0.48986826  0.02441878]


In [20]:
model_data

(           rt  response subj_idx         v         a         z         t
 0    1.358425       0.0     none -1.876421  2.356659  0.489868  0.024419
 1    0.912411       0.0     none -1.876421  2.356659  0.489868  0.024419
 2    0.888411       0.0     none -1.876421  2.356659  0.489868  0.024419
 3    1.012410       0.0     none -1.876421  2.356659  0.489868  0.024419
 4    1.510432       0.0     none -1.876421  2.356659  0.489868  0.024419
 ..        ...       ...      ...       ...       ...       ...       ...
 995  1.302423       0.0     none -1.876421  2.356659  0.489868  0.024419
 996  1.658439       0.0     none -1.876421  2.356659  0.489868  0.024419
 997  1.151415       0.0     none -1.876421  2.356659  0.489868  0.024419
 998  1.940452       0.0     none -1.876421  2.356659  0.489868  0.024419
 999  1.906451       0.0     none -1.876421  2.356659  0.489868  0.024419
 
 [1000 rows x 7 columns],
 {'v': -1.876421184995665,
  'a': 2.356659141427057,
  'z': 0.4898682565767404,
  't